In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.4 MB/s eta 0:00:00


In [ ]:
import spacy
from transformers import pipeline, BertForQuestionAnswering, BertTokenizer
import torch
import os

nlp = spacy.load("en_core_web_sm")

knowledge_graph = {}

# List of knowledge graph files
knowledge_graph_files = ['knowledge_graph.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt']

# Function to read and merge knowledge graph data from multiple files
def read_and_merge_knowledge_graph(files):
    merged_graph = {}
    for file in files:
        with open(file, 'r') as file:
            for line in file:
                line = line.strip().split('\t')
                entity = line[0]
                attribute = line[1]
                value = line[2]

                if entity not in merged_graph:
                    merged_graph[entity] = {}
                merged_graph[entity][attribute] = value
    return merged_graph

knowledge_graph = read_and_merge_knowledge_graph(knowledge_graph_files)

# Load the BERT-based question-answering model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def answer_question(question):
    # Process the question with spaCy
    doc = nlp(question)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    for entity, entity_type in entities:
        if entity in knowledge_graph:
            attributes = knowledge_graph[entity]
            for attribute, value in attributes.items():
                if attribute.lower() in question.lower():
                    return value

    # Check if the question is about the age of a player
    if "age" in question.lower() or "how old" in question.lower():
        for entity, attributes in knowledge_graph.items():
            if "date of birth" in attributes:
                dob = attributes["date of birth"]

                # Calculate age based on date of birth
                year_of_birth = int(dob.split("-")[0])
                current_year = 2023
                age = current_year - year_of_birth

                return f"{entity} is {age} years old."

    # Check if the question is about the youngest or oldest player (optional)
    if "youngest" in question.lower() or "oldest" in question.lower():
        target_attribute = "date of birth"
        target_player = None
        target_value = float('inf') if "youngest" in question.lower() else 0

        for entity, attributes in knowledge_graph.items():
            if target_attribute in attributes and "jersey" in attributes:
                dob = attributes[target_attribute]
                jersey = attributes["jersey"]

                # Calculate age based on date of birth
                year_of_birth = int(dob.split("-")[0])
                current_year = 2023
                age = current_year - year_of_birth

                if "youngest" in question.lower():
                    if age < target_value:
                        target_value = age
                        target_player = entity
                elif "oldest" in question.lower():
                    if age > target_value:
                        target_value = age
                        target_player = entity

        if target_player:
            return target_player

    # If the question doesn't match any of the conditions above, use the BERT-based model
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    start_positions, end_positions = model(**inputs)
    start_index = torch.argmax(start_positions)
    end_index = torch.argmax(end_positions)
    answer = text[start_index:end_index+1]

    return answer

question = "how many caps Diego Costa have?"
text = ""  # You should provide the context text here
answer = answer_question(question)
print("Question:", question)
print("Answer:", answer)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: how many caps Diego Costa have?
Answer: 31
